Importing the necessaries libraries to work with

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn import cluster

Reading the data

In [12]:
dataSet = pd.read_csv("Guilherme Antonio Aguiar Pessanha Santos - vehicle_1Mb.csv")
dataSet.head()

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,58589,7303354444,https://slo.craigslist.org/cto/d/nipomo-2008-f...,san luis obispo,https://slo.craigslist.org,7000,2008.0,ford,f150,good,...,NaN,pickup,NaN,https://images.craigslist.org/00w0w_abA4EveUZw...,This is a Green 2008 FORD F-150 XL 4 DOOR ext...,NaN,ca,350.298,-1.204.894,2021-04-07T14:28:18-0700
1,98736,7314126068,https://jacksonville.craigslist.org/cto/d/sain...,jacksonville,https://jacksonville.craigslist.org,17500,2016.0,bmw,328i,like new,...,mid-size,sedan,blue,https://images.craigslist.org/00e0e_fks755tyfI...,2016 Bmw 328i. 63000 miles. Second owner. Serv...,NaN,fl,300.815,-815.477,2021-04-28T20:14:28-0400
2,401980,7303685904,https://bellingham.craigslist.org/ctd/d/puyall...,bellingham,https://bellingham.craigslist.org,44999,2019.0,ram,1500,NaN,...,NaN,truck,white,https://images.craigslist.org/00U0U_iIvcoJZBwq...,Joydrive: 253-201-0804 https://joydrive.com/...,NaN,wa,471.991,-1.223.151,2021-04-08T09:17:55-0700
3,257546,7311166305,https://southjersey.craigslist.org/cto/d/egg-h...,south jersey,https://southjersey.craigslist.org,3990,2002.0,honda,cr-v,good,...,NaN,NaN,NaN,https://images.craigslist.org/01717_eNGBPppC61...,"2002 Honda CR-V 6 cylinder, FWD Clean title. ...",NaN,nj,393.741,-746.118,2021-04-22T21:15:18-0400
4,188630,7313149322,https://capecod.craigslist.org/ctd/d/cohoes-20...,cape cod / islands,https://capecod.craigslist.org,23900,2016.0,rover,discovery sport,excellent,...,full-size,SUV,black,https://images.craigslist.org/00R0R_7QyHgQgUd0...,2016 Land Rover Discovery Sport AWD 4dr HSE - ...,NaN,ma,42.777.828,-7.374.899,2021-04-26T21:23:07-0400


Seeing its info

In [13]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    304 non-null    int64  
 1   id            304 non-null    int64  
 2   url           304 non-null    object 
 3   region        304 non-null    object 
 4   region_url    304 non-null    object 
 5   price         304 non-null    int64  
 6   year          304 non-null    float64
 7   manufacturer  296 non-null    object 
 8   model         304 non-null    object 
 9   condition     192 non-null    object 
 10  cylinders     182 non-null    object 
 11  fuel          302 non-null    object 
 12  odometer      297 non-null    float64
 13  title_status  295 non-null    object 
 14  transmission  301 non-null    object 
 15  VIN           200 non-null    object 
 16  drive         210 non-null    object 
 17  size          89 non-null     object 
 18  type          243 non-null    

The features of numbers 0, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 24, 25 have data missing init. I will remove the feature of number 2 because it only has 1 non-Null sata init

Discrbing the features

In [14]:
dataSet.describe()

,Unnamed: 0,id,price,year,odometer,county
count,304.000000,3.040000e+02,304.000000,304.000000,2.970000e+02,0.0
mean,207976.680921,7.311443e+09,17330.210526,2011.105263,9.470059e+04,NaN
std,119573.752387,4.503619e+06,15122.034717,10.597750,8.684085e+04,NaN
min,2933.000000,7.301662e+09,0.000000,1923.000000,1.000000e+00,NaN
25%,98635.500000,7.307754e+09,5971.250000,2009.000000,3.890700e+04,NaN
50%,208630.000000,7.312816e+09,13998.000000,2014.000000,8.700000e+04,NaN
75%,311334.750000,7.315379e+09,25990.000000,2017.000000,1.320000e+05,NaN
max,426484.000000,7.317033e+09,90500.000000,2021.000000,1.111111e+06,NaN


The feature of number 22 which doesn't have any data init 

In [17]:
dataSet["county"]

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
299   NaN
300   NaN
301   NaN
302   NaN
303   NaN
Name: county, Length: 304, dtype: float64

In [20]:
dataSet["size"]

0            NaN
1       mid-size
2            NaN
3            NaN
4      full-size
         ...    
299          NaN
300          NaN
301          NaN
302          NaN
303          NaN
Name: size, Length: 304, dtype: object

The feature is county and only has one non-null data, so now i will remove the the feature from the dataSet 

In [21]:
dataSet["size"].keys()


RangeIndex(start=0, stop=304, step=1)

In [25]:
dataSet_cleaned = dataSet.copy()
# dataSet1.drop("county",axis = 22)
# deleting the irrelevants features
dataSet_cleaned.drop(["county", "size", "id", "url", "region_url", "VIN", "image_url", "description", "lat", "long", "posting_date"], axis = 1, inplace= True)
dataSet_cleaned

,Unnamed: 0,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,state
0,58589,san luis obispo,7000,2008.0,ford,f150,good,NaN,gas,265000.0,clean,automatic,NaN,pickup,NaN,ca
1,98736,jacksonville,17500,2016.0,bmw,328i,like new,4 cylinders,gas,63030.0,clean,automatic,rwd,sedan,blue,fl
2,401980,bellingham,44999,2019.0,ram,1500,NaN,8 cylinders,gas,27291.0,clean,automatic,4wd,truck,white,wa
3,257546,south jersey,3990,2002.0,honda,cr-v,good,NaN,gas,136230.0,clean,automatic,NaN,NaN,NaN,nj
4,188630,cape cod / islands,23900,2016.0,rover,discovery sport,excellent,4 cylinders,gas,60412.0,clean,automatic,4wd,SUV,black,ma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,80436,fort collins / north CO,24900,2016.0,ford,f250 super duty,excellent,8 cylinders,gas,147635.0,clean,automatic,4wd,pickup,white,co
300,58144,san luis obispo,14777,2015.0,volvo,xc60 local lots of records,excellent,NaN,gas,118247.0,clean,automatic,fwd,SUV,NaN,ca
301,176811,new orleans,7200,2012.0,toyota,corolla,excellent,4 cylinders,gas,170000.0,clean,automatic,fwd,NaN,NaN,la
302,422865,milwaukee,3600,1995.0,ford,f-150,excellent,8 cylinders,gas,199000.0,clean,automatic,rwd,truck,purple,wi


Let's check the info of dataSet_cleaned

In [35]:
print(dataSet_cleaned.info())
print(dataSet_cleaned["manufacturer"].value_counts())
print(dataSet_cleaned["condition"].value_counts())
print(dataSet_cleaned["cylinders"].value_counts())
print(dataSet_cleaned["fuel"].value_counts())
print(dataSet_cleaned["odometer"].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    304 non-null    int64  
 1   region        304 non-null    object 
 2   price         304 non-null    int64  
 3   year          304 non-null    float64
 4   manufacturer  296 non-null    object 
 5   model         304 non-null    object 
 6   condition     192 non-null    object 
 7   cylinders     182 non-null    object 
 8   fuel          302 non-null    object 
 9   odometer      297 non-null    float64
 10  title_status  295 non-null    object 
 11  transmission  301 non-null    object 
 12  drive         210 non-null    object 
 13  type          243 non-null    object 
 14  paint_color   209 non-null    object 
 15  state         304 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 38.1+ KB
None
ford             43
chevrolet        42
nissan      

In [36]:
print(dataSet_cleaned["title_status"].value_counts())
print(dataSet_cleaned["transmission"].value_counts())
print(dataSet_cleaned["drive"].value_counts())
print(dataSet_cleaned["type"].value_counts())
print(dataSet_cleaned["paint_color"].value_counts())

clean      285
rebuilt      8
lien         1
salvage      1
Name: title_status, dtype: int64
automatic    242
other         39
manual        20
Name: transmission, dtype: int64
4wd    85
fwd    78
rwd    47
Name: drive, dtype: int64
sedan          58
SUV            50
truck          37
pickup         32
wagon          13
hatchback      13
coupe          13
other          11
van             7
convertible     6
mini-van        3
Name: type, dtype: int64
white     60
black     44
blue      30
grey      21
silver    20
red       14
custom     7
brown      6
green      3
orange     3
purple     1
Name: paint_color, dtype: int64


In [13]:
def split_train_test(data, test_ratio):
  "splits the data"
  shuffled_indices = np.random.permutation(len(data))
  test_set_size = int(len(data) * test_ratio)
  test_indices = shuffled_indices[:test_set_size]
  train_indices = shuffled_indices[test_set_size:]
  return data.iloc[train_indices], data.iloc[test_indices]

In [16]:
 train_set, test_set = split_train_test(dataSet, 0.2)
 print(len(train_set))
 print(len(test_set))
 test_set.head()

244
61


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
23,221108.0,7307499641,https://loz.craigslist.org/cto/d/macks-creek-r...,lake of the ozarks,https://loz.craigslist.org,29000,2017.0,jeep,wrangler unlimited,good,...,NaN,SUV,blue,https://images.craigslist.org/00O0O_1KpjdiA0fb...,"2017 Jeep Wrangler · Suv · Driven 95,000 miles...",NaN,mo,379.616,-929.603,2021-04-15T19:20:13-0500
175,245372.0,7316220734,https://lincoln.craigslist.org/ctd/d/columbus-...,lincoln,https://lincoln.craigslist.org,28888,2018.0,lincoln,mkc,NaN,...,NaN,SUV,NaN,https://images.craigslist.org/00Z0Z_ghDK5BIHT7...,"2018 Lincoln MKC Select AWD -- $28,888 ☎...",NaN,ne,41.438.366,-97.316.899,2021-05-03T10:35:07-0500
167,358067.0,7306800052,https://memphis.craigslist.org/ctd/d/memphis-2...,memphis,https://memphis.craigslist.org,10995,2012.0,toyota,camry,NaN,...,NaN,sedan,white,https://images.craigslist.org/00I0I_7HMlYjXP9E...,"2012 *Toyota* *Camry* Sedan - $10,995Call Us ...",NaN,tn,35.191.437,-89.894.973,2021-04-14T13:30:21-0500
290,93226.0,7305668893,https://daytona.craigslist.org/ctd/d/orlando-2...,daytona beach,https://daytona.craigslist.org,1,2017.0,ford,f-150,excellent,...,full-size,truck,white,https://images.craigslist.org/00v0v_4adamixylj...,CREDIT AMNESTY! SE HABLA ESPANOL !!OPEN SUNDA...,NaN,fl,28.466.088,-81.396.429,2021-04-12T12:15:01-0400
291,85572.0,7304630826,https://hartford.craigslist.org/ctd/d/hartford...,hartford,https://hartford.craigslist.org,24990,2017.0,lexus,es 350 sedan 4d,good,...,NaN,sedan,NaN,https://images.craigslist.org/00f0f_MGjKA4gkdk...,Carvana is the safer way to buy a car During t...,NaN,ct,41.76,-72.68,2021-04-10T09:51:21-0400
